# Выберем лучшую из двух моделей по предсказанию доверительных интвералов
- Catboost и одинаковые интервалы
- NGBoost

In [63]:
import pandas as pd
import numpy as np
import pickle

from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor, Pool

In [8]:
SEED = 42

In [9]:
df = pd.read_csv("../data/processed/clean_data.csv")

In [12]:
df_train, df_test = train_test_split(df, shuffle=True, random_state=SEED)
y_train = df_train.priceLog
y_test = df_test.priceLog
X_train = df_train.drop("priceLog", axis=1)
X_test = df_test.drop("priceLog", axis=1)

In [54]:
def accuracy_ci_score(y_real, min_values, max_values):
    acc = np.mean(((y_real > min_values) & (y_real < max_values)))
    range = np.mean(max_values - min_values)
    return acc, range

# CatBoost

In [43]:
cat_boost_path = "../models/cagboost_model.bin"
model = CatBoostRegressor().load_model(cat_boost_path)

In [44]:
preds = model.predict(X_test)

In [62]:
for shift in [0.000, 0.01, 0.2, 0.3, 0.35, 0.4, 0.5, 1]:
    v, r = accuracy_ci_score(y_test, preds-shift, preds+shift)
    print(f"{shift} = acc = {round(v,2)}, ci range = {round(r,2)}")

0.0 = acc = 0.0, ci range = 0.0
0.01 = acc = 0.08, ci range = 0.02
0.2 = acc = 0.7, ci range = 0.4
0.3 = acc = 0.89, ci range = 0.6
0.35 = acc = 0.93, ci range = 0.7
0.4 = acc = 0.96, ci range = 0.8
0.5 = acc = 0.98, ci range = 1.0
1 = acc = 1.0, ci range = 2.0


Можно для точности угадывания 93-96 процентов мы можем взять предсазание логарифм цены +- 0.35 (разброс 0.7)

# NGBoost

In [74]:
ng_boost_path = "../models/ngboost_model.pkl"
with open(ng_boost_path, "rb") as f:
    model, dv = pickle.load(f)

In [75]:
def get_ci(model: NGBRegressor, X: np.array) -> np.array:
    """Get predictions, and 95% confidence interval for X"""
    z_score = 1.95  # 95% confidence interval
    preds = model.pred_param(X)
    mean = preds[:, 0]
    lower = preds[:, 0] - z_score * np.exp(preds[:, 1])
    upper = preds[:, 0] + z_score * np.exp(preds[:, 1])
    return np.c_[mean, lower, upper]

In [77]:
X_for_ngboost = dv.transform(X_test.to_dict(orient="records"))

In [78]:
preds = get_ci(model, X_for_ngboost)

In [81]:
v, r = accuracy_ci_score(y_test, preds[:,1], preds[:,2])
print(f"ngboost = acc = {round(v,2)}, ci range = {round(r,2)}")

ngboost = acc = 0.96, ci range = 0.59


Для NGboost диапазон предсказывается точнее, и для 96% попадания в доверительный интервал средний разброс 0.59 (+-0.3)

# Вывод

Будем использовать модель NGBoost, т.к. доверительный интервал у нее получается считать точнее.  
Хотя ошибка все равно достаточно большая, и возможн нам стоить уменьшить интервал (брать не 95%, а 80% или может 60%)  